In [ ]:
!pip install transformers text-hammer pyreadstat

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup , AutoTokenizer, TFAutoModel, TFRobertaModel
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, f1_score
import matplotlib.pyplot as plt
import os
import io
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold
from sklearn import svm
from collections import defaultdict
import text_hammer as th
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import transformers
import random as rd
import keras.backend as K
from numpy.random import seed
from tensorflow.keras import layers
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import multilabel_confusion_matrix
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# identify and specify the GPU as the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

### Functions

In [ ]:
#------------------------STEP 1 --------------------------------#

# Metric1 construction to compute the % of success tweet by tweet

def metric1(y_true, y_pred):
    n = K.cast(K.shape(y_true)[0], 'float32')
    y_pred_r = K.round(y_pred)
    res = K.all(K.equal(y_true, y_pred_r), axis=1)
    res = K.cast(res, 'float32')
    res = K.sum(res) / n
    return res

#------------------------STEP 2 --------------------------------#

# Metric2 construction to compute the % of success label by label
# (equal to binary-accuracy)

def metric2(y_true, y_pred):
    n = K.cast(K.shape(y_true)[0], 'float32')
    y_pred_r = K.round(y_pred)
    res = abs(y_true - y_pred_r)
    count_error = K.sum(K.cast(K.equal(res, 1), 'float32'),axis=0)
    res = 1-count_error / n
    res = K.mean(res)
    return res

In [ ]:
def get_clean(x):
    mention = r'@\w+'
    hash = r'#\w+'
    x = str(x).lower().replace('\\', '').replace('_', '')
    x = re.sub(r'[^\x00-\x7F]+', ' ', x)
    x = th.cont_exp(x)
    x = th.remove_emails(x)
    x = th.remove_urls(x)
    x = re.sub(mention, ' ', x)
    x = re.sub(hash, ' ', x)
    x = th.remove_html_tags(x)
    x = th.remove_rt(x)
    x = th.remove_accented_chars(x)
    x = th.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    x = re.sub(r'\s+', ' ', x).strip()
    x = re.sub(r'\w*\d+\w*', ' ', x).strip()
    return x

### Data

In [ ]:
df = pd.read_spss("/content/drive/MyDrive/VA_EN_TU_2012-2020_3000_tweets_relevant_V03_labeled_1200_cleaned.sav")
data = df[['text', 'Label_A2_negative', 'Label_A3_neutral','Label_A1_positive']].copy()
data.head()

In [ ]:
data['cleaned_text'] = data['text'].apply(get_clean)

In [ ]:
## Individual Label Distribution
label_sums = data[['Label_A2_negative', 'Label_A3_neutral','Label_A1_positive']].sum(axis=0)
labels = ['Label_A2_negative', 'Label_A3_neutral','Label_A1_positive']

# label_counts = data[['label_1_positive', 'label_2_negative', 'label_3_neutral']].sum()
# print(label_counts)

plt.figure(figsize=(10,6))
plt.bar(labels, label_sums)
plt.ylabel('Number of tweets')
plt.title('Individual Label Distribution')
plt.show()

## Label Combinations Distribution
df_train_labels = pd.DataFrame(data[['Label_A2_negative', 'Label_A3_neutral','Label_A1_positive']], columns=labels)
combinations = df_train_labels.groupby(labels).size().reset_index().rename(columns={0:'count'})


combinations = combinations.sort_values(by='count', ascending=False) # by count to get most frequent combinations
combinations['Label Combination'] = combinations[labels].astype(int).astype(str).agg(','.join, axis=1)

plt.figure(figsize=(12,8))
combinations.plot(x='Label Combination', y='count', kind='bar', legend=False)
plt.title('Label Combinations Distribution')
plt.ylabel('Number of tweets')
plt.xticks(rotation=45)
plt.show()


### BERTweet model

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow import keras

# Load tokenizer and model
pretrained_model_name = 'vinai/bertweet-large'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
pretrained_model = TFAutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=3)

# Define the model
def get_model():

    seq_len = 128  # Define your sequence length

    input_ids = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name='attention_mask')

    output = pretrained_model(input_ids, attention_mask=attention_mask)[0]  # Using [0] to get the last hidden states

    # Flatten the output or use global pooling
    flat_output = tf.keras.layers.Flatten()(output)

    # Add a dense layer for classification
    predictions = tf.keras.layers.Dense(3, activation='sigmoid')(flat_output)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=predictions)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])  # You can change the metric as needed

    return model

# Model summary
model = get_model()
model.summary()


In [ ]:
def fit_the_model(model, X_tr, y_tr, X_val, y_val, num):
    callbacks = [
        EarlyStopping(monitor="val_accuracy", patience=earlyStopPatience, mode="max"),
        ModelCheckpoint(filepath="MODEL_BERTweet/best_model"+str(num)+".hdf5", monitor="val_accuracy", mode='max', save_best_only=True, verbose=1)
    ]
    model.fit(X_tr, y_tr, validation_data = (X_val,y_val), epochs=nb_epoch, verbose=1, callbacks=callbacks, batch_size=batch_size)
    return model

In [ ]:
list_of_models = []
list_history = []

kfold = KFold(n_splits=nb_fold, shuffle=True, random_state=42)

# Training with k-fold method
k = 1
for train_index, val_index in kfold.split(X_ids_train,Y_train):
    print(f'\nTraining model {k}...')
    model = get_model()
    history = fit_the_model(model,
                            [X_ids_train[train_index],
                             X_mask_train[train_index]],
                            Y_train[train_index],
                            [X_ids_train[val_index],
                             X_mask_train[val_index]],
                            Y_train[val_index], k)
    list_of_models.append(history)
    list_history.append(history.history.history)
    k += 1

print("---Finished---")

In [ ]:
def ConfusionMatrix(y_true , y_pred):
    cm = confusion_matrix(y_true , y_pred)
    return cm/y_true.shape[0]

def metric2_per_class(y_true,y_pred):
    L = []
    y_pred = y_pred.reshape((len(y_pred),1))
    for i in range(0,3):
        L.append(y_true[(y_true==y_pred) & (y_true==i)].shape[0] / y_true[(y_true==i)].shape[0])
    return L

In [ ]:
for i in range(nb_fold):
    print(50*'-',f"MODEL PERFORMANCE {i+1}",50*'-')

    average_loss = list_history[i]['loss']
    average_val_loss = list_history[i]['val_loss']

    average_metric_1 = list_history[i]['accuracy']
    average_val_metric_1 = list_history[i]['val_accuracy']

    plt.figure(figsize=(16,4))

    txtTitres = ["Training and validation loss","Training and validation accuracy"]
    txtYLabel = ["Loss","Metric 1 "]

    L1 = [average_loss, average_metric_1]
    L2 = [average_val_loss, average_val_metric_1]

    for k in range(2):
        if len(L1[0]) < nb_epoch:
            nb_epoch_plot=len(L1[0])
        else:
            nb_epoch_plot=nb_epoch
        Epochs = range(1, nb_epoch_plot+1)
        plt.subplot(1,3,k+1)
        plt.plot(Epochs, L1[k], color='b', marker='o', label="training")
        plt.plot(Epochs, L2[k], color='r', marker='o', label="validation")
        plt.grid()
        plt.title(txtTitres[k])
        plt.xlabel("Epochs")
        plt.ylabel(txtYLabel[k])
        plt.legend()
    plt.show()

In [ ]:
L_best_models = []
for i in range(nb_fold):
    path = "MODEL_BERTweet/best_model" + str(i+1) + ".hdf5"
    L_best_models.append(keras.models.load_model(path,
                                                 custom_objects={"TFRobertaModel": pretrained_model},
                                                 compile=False))

In [ ]:
L_predictions = []
L_cm = []
L_f1 = []
L_m1, L_m2 = [], []

k = 0

kfold = KFold(n_splits=nb_fold, shuffle=True, random_state=42)
for train_index, val_index in kfold.split(X_ids_train,Y_train):

    prediction = np.argmax(L_best_models[k].predict([X_ids_train[val_index], X_mask_train[val_index]]), axis=1)

    cm = ConfusionMatrix(Y_train[val_index] , prediction)
    f1 = f1_score(Y_train[val_index] , prediction, average=None)
    m1 = metric1(Y_train[val_index], prediction)
    m2 = metric2_per_class(Y_train[val_index] , prediction)

    L_predictions.append(prediction)
    L_cm.append(cm)
    L_f1.append(f1)
    L_m1.append(m1)
    L_m2.append(m2)
    k += 1

In [ ]:
print(40*'-',f"SUMMARY OF MODEL PERFORMANCE ON VALIDATION DATA",20*'-',"\n")

m1 = np.mean(L_m1,axis=0)
m2 = np.mean(L_m2,axis=0)

print("metric1 validation mean :", m1, "\n")
print("metric2 validation mean per class :", m2, "\n")

print("Average F1 score on positive/negative/neutral: ", np.mean(L_f1,axis=0))

avg_cm = np.mean(L_cm,axis=0)
plt.figure(figsize=(5,5))
ax = sns.heatmap(avg_cm, annot=True, fmt='.2%' , cmap="Blues")
ax.set_xlabel('\n Predicted Values')
ax.set_ylabel('Actual Values ')
ax.set_title(f'Confusion Matrix of the model ')
ax.xaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
ax.yaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
plt.show()

In [ ]:
L_pred = [np.round(model.predict([X_ids_test, X_mask_test])) for model in L_best_models]
prediction = np.median(L_pred, axis=0)

predicted_labels = np.argmax(prediction, axis=1).reshape(-1, 1)

In [ ]:
print("Classification Report: \n", classification_report(Y_test, predicted_labels,target_names=['Negative','Neutral','Positive']))

In [ ]:
cm = ConfusionMatrix(Y_test , predicted_labels)
f1 = f1_score(Y_test , predicted_labels, average=None)
m1 = float(metric1(Y_test, predicted_labels))
m2 = float(metric2_2(Y_test, predicted_labels))

print("metric1 Test mean :", m1, "\n")
print("metric2 Test mean :", m2, "\n")
print("Average F1 score on negative/neutral/positive: ", np.mean(f1,axis=0))

plt.figure(figsize=(5,5))
ax = sns.heatmap(cm, annot=True, fmt='.2%' , cmap="Blues")
ax.set_xlabel('\n Predicted Values')
ax.set_ylabel('Actual Values ')
ax.set_title(f'Confusion Matrix of the model ')
ax.xaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
ax.yaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
plt.show()